In [77]:
import pandas as pd
import nltk
import string
import pickle
import re

import emoji

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# gensim
from gensim import corpora, models, similarities, matutils
import pyLDAvis.gensim
# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
df = pd.read_csv('data/IRAhandle_tweets_1.csv')

In [3]:
df.shape

(381016, 15)

In [4]:
df.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll


In [5]:
df['account_category'].unique()

array(['RightTroll', 'NonEnglish', 'Fearmonger', 'LeftTroll', 'Unknown',
       'HashtagGamer', 'NewsFeed', 'Commercial'], dtype=object)

In [84]:
df_rtrolls = df[df['account_category'] == 'RightTroll']

In [85]:
df_rtrolls.shape

(128681, 15)

In [79]:

''.join(c for c in str if c in emoji.UNICODE_EMOJI)

TypeError: 'type' object is not iterable

In [78]:
# remove links
def remove_links(string):
    return re.sub(r'(https?:\/\/)(\s)?(www\.)?(\s?)(\w+\.)*([\w\-\s]+\/)*([\w\-\?=\&]+)\/?', '', string)

In [79]:
df_rtrolls['content'] = df_rtrolls['content'].apply(remove_links)

/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [91]:
def custom_tokenizer(text):
    full_punc = '’‘“”...–…�' + string.punctuation
    # remove punctuation
    remove_punct = str.maketrans('', '', full_punc)
    text = text.translate(remove_punct)

    # remove digits and convert to lower case
    remove_digits = str.maketrans('', '', string.digits)
    text = text.lower().translate(remove_digits)

    # tokenize
    tokens = word_tokenize(text)

    # remove stop words
    punc = [str(i) for i in string.punctuation]
    stop_words = ['rt', 'retweet'] + stopwords.words('english')
    tokens_stop = [y for y in tokens if y not in stop_words]

    # stem
    stemmer = SnowballStemmer('english')
    tokens_stem = [stemmer.stem(y) for y in tokens_stop] 

    return tokens_stop

In [92]:
cv = CountVectorizer(tokenizer=custom_tokenizer)

In [93]:
df_rtrolls.content.head()

0    "We have a sitting Democrat US Senator on tria...
1    Marshawn Lynch arrives to game in anti-Trump s...
2    Daughter of fallen Navy Sailor delivers powerf...
3    JUST IN: President Trump dedicates Presidents ...
4    19,000 RESPECTING our National Anthem! #StandF...
Name: content, dtype: object

In [94]:
# Create the term-document matrix# Creat 
# Transpose it so the terms are the rows
counts = cv.fit_transform(df_rtrolls.content).transpose()

In [95]:
tokens = counts.transpose()

##### Convert to gensim
We need to convert our sparse `scipy` matrix to a `gensim`-friendly object called a Corpus:

In [96]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(counts)

##### Map matrix rows to words (tokens)
We need to save a mapping (dict) of row id to word (token) for later use by gensim:

In [97]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
len(id2word)

83563

In [63]:
list(cv.vocabulary_.keys())

['sitting',
 'democrat',
 'us',
 'senator',
 'trial',
 'corruption',
 'youve',
 'barely',
 'heard',
 'peep',
 'mainstream',
 'media',
 'nedryun',
 'httpstcoghgdoic',
 'marshawn',
 'lynch',
 'arrives',
 'game',
 'antitrump',
 'shirt',
 'judging',
 'sagging',
 'pants',
 'say',
 'vs',
 'belt',
 'httpstcomlhilzz',
 'daughter',
 'fallen',
 'navy',
 'sailor',
 'delivers',
 'powerful',
 'monologue',
 'anthem',
 'protests',
 'burns',
 'nfl',
 'packers',
 'gear',
 'boycottnfl',
 'httpstcoqdlfbgmeag',
 'president',
 'trump',
 'dedicates',
 'presidents',
 'cup',
 'golf',
 'tournament',
 'trophy',
 'people',
 'florida',
 'texas',
 'puerto',
 'rico',
 'httpstcozwvadjae',
 'respecting',
 'national',
 'standforouranthem🇺🇸',
 'httpstcoczutygamqv',
 'dan',
 'bongino',
 'nobody',
 'trolls',
 'liberals',
 'better',
 'donald',
 'exactly',
 'httpstcoaigvacj',
 '🐝🐝🐝',
 'httpstcomorlaqwz',
 'senatormenendez',
 'carmenyulincruz',
 'doesnt',
 'matter',
 'cnn',
 'report',
 'crimes',
 'wont',
 'change',
 'fact',

In [98]:
g_dict = corpora.Dictionary([list(cv.vocabulary_.keys())])

2018-08-16 17:41:30,446 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-16 17:41:30,788 : INFO : built Dictionary(83563 unique tokens: ['aa', 'aaa', 'aaaaaaaaaaaaand', 'aaaaaaaamen', 'aaaaaaaand']...) from 1 documents (total 83563 corpus positions)


## LDA
At this point we can simply plow ahead in creating an LDA model.  It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

In [102]:
for tweet in df_rtrolls.content:
    if 'rubio' in tweet:
        print(tweet)

.@marcorubio, the Libyan people toppled Qaddafi? The jihadists you urged us to arm toppled Qaddafi .. And now they're running the country.
 @marcorubio @tedcruz @realDonaldTrump @maggieNYT @nytpolitics @JohnBranchNYT @BrentNYT @nytimesworld @MHPshow
America if #Trump loses.... #WhiteGenocide #gop #tcot #cruzcrew #rubio #cruz #tlot #ccot 
.@marcorubio says @realDonaldTrump's debate no-show may have helped him in Iowa  
RT MrsScottBaio: Why didn't marcorubio SenJohnMcCain or Hatch bother to tell the DNC &amp; Obama to denounce Antifa &amp; BLM?
RT LyndaG1963: THIS is who you stand with marcorubio ?? Good to know!! You will never be Prez &amp; now your reelectio… 
RT pgunnels: Watching communist sympathizer marcorubio make common cause with communist thugs wielding clubs and bricks demands removal fro…
RT SonofLiberty357: marcorubio so this is the side you've chosen? 
RT ScottBaio: Did marcorubio MittRomney or SenJohnMcCain condemn the senator of Missouri 4 wanting 2 assassinate Pres real

In [99]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=30)

2018-08-16 17:41:30,812 : INFO : using symmetric alpha at 0.05
2018-08-16 17:41:30,813 : INFO : using symmetric eta at 0.05
2018-08-16 17:41:30,829 : INFO : using serial LDA version on this node
2018-08-16 17:41:31,001 : INFO : running online (multi-pass) LDA training, 20 topics, 30 passes over the supplied corpus of 128681 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2018-08-16 17:41:31,010 : INFO : PROGRESS: pass 0, at document #2000/128681
2018-08-16 17:41:31,606 : INFO : merging changes from 2000 documents into a model of 128681 documents
2018-08-16 17:41:31,743 : INFO : topic #6 (0.050): 0.028*"abiss" + 0.028*"abi" + 0.025*"acapa" + 0.012*"look" + 0.011*"white" + 0.011*"breaking" + 0.009*"trump" + 0.009*"house" + 0.009*"video" + 0.009*"obama"
2018-08-16 17:41:31,745 : INFO : topic #2 (0.050): 0.027*"trump" + 0.017*"acapa" + 0.008*"video" + 0.006*"reporter" + 0.006*"abi" + 0.

2018-08-16 17:41:34,604 : INFO : PROGRESS: pass 0, at document #14000/128681
2018-08-16 17:41:34,990 : INFO : merging changes from 2000 documents into a model of 128681 documents
2018-08-16 17:41:35,118 : INFO : topic #17 (0.050): 0.049*"enlist" + 0.036*"read" + 0.024*"truth" + 0.021*"usfa" + 0.020*"want" + 0.016*"stand" + 0.014*"think" + 0.014*"another" + 0.013*"mt" + 0.013*"dont"
2018-08-16 17:41:35,119 : INFO : topic #1 (0.050): 0.024*"enlist" + 0.022*"god" + 0.018*"american" + 0.015*"vote" + 0.013*"bless" + 0.013*"patriots" + 0.013*"kim" + 0.011*"us" + 0.011*"usfa" + 0.009*"never"
2018-08-16 17:41:35,121 : INFO : topic #9 (0.050): 0.031*"enlist" + 0.024*"need" + 0.018*"support" + 0.018*"freedom" + 0.017*"us" + 0.015*"gave" + 0.014*"usfa" + 0.013*"thats" + 0.012*"believe" + 0.011*"thank"
2018-08-16 17:41:35,123 : INFO : topic #19 (0.050): 0.165*"army" + 0.052*"enlist" + 0.019*"people" + 0.015*"blm" + 0.011*"read" + 0.011*"awaits" + 0.011*"socialism" + 0.011*"black" + 0.010*"america"

2018-08-16 17:41:38,857 : INFO : topic #10 (0.050): 0.019*"go" + 0.017*"time" + 0.013*"hes" + 0.011*"family" + 0.010*"much" + 0.010*"lets" + 0.010*"trumpbecause" + 0.009*"th" + 0.008*"something" + 0.008*"people"
2018-08-16 17:41:38,859 : INFO : topic #2 (0.050): 0.013*"america" + 0.012*"amb" + 0.012*"stand" + 0.010*"work" + 0.010*"leader" + 0.010*"dont" + 0.009*"gets" + 0.009*"thing" + 0.009*"get" + 0.008*"already"
2018-08-16 17:41:38,861 : INFO : topic #18 (0.050): 0.031*"foxnews" + 0.027*"make" + 0.021*"realdonaldtrump" + 0.021*"im" + 0.015*"democrats" + 0.014*"day" + 0.014*"never" + 0.013*"police" + 0.012*"darlovesamerica" + 0.011*"best"
2018-08-16 17:41:38,864 : INFO : topic #15 (0.050): 0.222*"amb" + 0.011*"trump" + 0.010*"say" + 0.010*"every" + 0.009*"watch" + 0.009*"lol" + 0.009*"alis" + 0.009*"lis" + 0.007*"mitchellvii" + 0.007*"seanhannity"
2018-08-16 17:41:38,866 : INFO : topic diff=0.233372, rho=0.277350
2018-08-16 17:41:38,878 : INFO : PROGRESS: pass 0, at document #28000/1

2018-08-16 17:41:41,875 : INFO : topic #15 (0.050): 0.130*"amb" + 0.017*"say" + 0.012*"trump" + 0.012*"every" + 0.011*"corrupt" + 0.010*"needs" + 0.010*"watch" + 0.009*"bernie" + 0.009*"seanhannity" + 0.008*"million"
2018-08-16 17:41:41,877 : INFO : topic #14 (0.050): 0.015*"hillary" + 0.014*"party" + 0.012*"made" + 0.011*"doesnt" + 0.011*"years" + 0.009*"got" + 0.008*"days" + 0.008*"leave" + 0.008*"bad" + 0.008*"oh"
2018-08-16 17:41:41,879 : INFO : topic diff=0.208774, rho=0.229416
2018-08-16 17:41:42,514 : INFO : -12.193 per-word bound, 4683.6 perplexity estimate based on a held-out corpus of 2000 documents with 20042 words
2018-08-16 17:41:42,515 : INFO : PROGRESS: pass 0, at document #40000/128681
2018-08-16 17:41:42,845 : INFO : merging changes from 2000 documents into a model of 128681 documents
2018-08-16 17:41:42,964 : INFO : topic #8 (0.050): 0.027*"fbi" + 0.019*"clinton" + 0.014*"hillary" + 0.013*"hillarys" + 0.012*"msm" + 0.011*"trump" + 0.010*"law" + 0.009*"lies" + 0.009*"o

2018-08-16 17:41:45,399 : INFO : topic #0 (0.050): 0.041*"news" + 0.022*"fake" + 0.018*"trump" + 0.018*"youtube" + 0.017*"cnn" + 0.012*"israel" + 0.011*"clinton" + 0.011*"children" + 0.010*"isis" + 0.010*"wants"
2018-08-16 17:41:45,401 : INFO : topic diff=0.225340, rho=0.200000
2018-08-16 17:41:45,413 : INFO : PROGRESS: pass 0, at document #52000/128681
2018-08-16 17:41:45,728 : INFO : merging changes from 2000 documents into a model of 128681 documents
2018-08-16 17:41:45,852 : INFO : topic #1 (0.050): 0.029*"vote" + 0.020*"god" + 0.015*"american" + 0.012*"last" + 0.010*"always" + 0.010*"wrong" + 0.010*"un" + 0.010*"people" + 0.009*"us" + 0.009*"full"
2018-08-16 17:41:45,854 : INFO : topic #14 (0.050): 0.016*"party" + 0.015*"years" + 0.013*"made" + 0.011*"care" + 0.011*"doesnt" + 0.010*"syria" + 0.009*"health" + 0.009*"amp" + 0.008*"r" + 0.008*"bad"
2018-08-16 17:41:45,856 : INFO : topic #17 (0.050): 0.031*"want" + 0.024*"dont" + 0.018*"think" + 0.012*"read" + 0.011*"mt" + 0.010*"done

...

2018-08-16 17:55:48,958 : INFO : topic #17 (0.050): 0.050*"ame" + 0.036*"want" + 0.029*"dont" + 0.020*"think" + 0.019*"truth" + 0.017*"see" + 0.012*"keep" + 0.012*"job" + 0.012*"dnc" + 0.011*"read"
2018-08-16 17:55:48,960 : INFO : topic #6 (0.050): 0.029*"white" + 0.020*"house" + 0.015*"muslim" + 0.012*"obama" + 0.012*"trumptrain" + 0.011*"potus" + 0.011*"group" + 0.011*"antifa" + 0.010*"year" + 0.010*"soros"
2018-08-16 17:55:48,963 : INFO : topic #4 (0.050): 0.084*"trump" + 0.035*"media" + 0.019*"tcot" + 0.017*"hillary" + 0.016*"donald" + 0.014*"supporters" + 0.011*"antifa" + 0.010*"dems" + 0.009*"president" + 0.008*"rally"
2018-08-16 17:55:48,965 : INFO : topic #16 (0.050): 0.046*"trump" + 0.032*"anche" + 0.031*"aamy" + 0.028*"president" + 0.010*"yes" + 0.009*"post" + 0.008*"chelsea" + 0.008*"wow" + 0.008*"military" + 0.008*"steph"
2018-08-16 17:55:48,967 : INFO : topic #18 (0.050): 0.028*"im" + 0.026*"make" + 0.024*"foxnews" + 0.018*"democrats" + 0.017*"never" + 0.016*"day" + 0.016*

2018-08-16 17:55:51,701 : INFO : topic #15 (0.050): 0.016*"amb" + 0.014*"trump" + 0.014*"seanhannity" + 0.014*"say" + 0.013*"every" + 0.012*"million" + 0.011*"lol" + 0.011*"fraud" + 0.011*"mad" + 0.010*"gop"
2018-08-16 17:55:51,704 : INFO : topic #4 (0.050): 0.084*"trump" + 0.036*"media" + 0.017*"supporters" + 0.013*"antifa" + 0.013*"donald" + 0.013*"hillary" + 0.012*"dems" + 0.010*"tcot" + 0.010*"president" + 0.008*"new"
2018-08-16 17:55:51,706 : INFO : topic #3 (0.050): 0.051*"trump" + 0.034*"charlottesville" + 0.028*"breaking" + 0.025*"look" + 0.024*"flig" + 0.020*"andrrre" + 0.019*"andris" + 0.019*"angiis" + 0.019*"video" + 0.014*"anderr"
2018-08-16 17:55:51,709 : INFO : topic #9 (0.050): 0.025*"says" + 0.024*"support" + 0.023*"need" + 0.018*"thats" + 0.018*"man" + 0.016*"ever" + 0.016*"thank" + 0.015*"right" + 0.013*"life" + 0.012*"believe"
2018-08-16 17:55:51,711 : INFO : topic diff=0.083502, rho=0.102956
2018-08-16 17:55:51,789 : INFO : PROGRESS: pass 29, at document #86000/1286

2018-08-16 17:55:54,016 : INFO : topic #13 (0.050): 0.049*"hillary" + 0.041*"clinton" + 0.016*"breaking" + 0.013*"didnt" + 0.013*"bill" + 0.011*"history" + 0.011*"islam" + 0.011*"could" + 0.011*"statue" + 0.010*"liberals"
2018-08-16 17:55:54,018 : INFO : topic #8 (0.050): 0.020*"fbi" + 0.017*"breaking" + 0.015*"attack" + 0.012*"comey" + 0.011*"lynch" + 0.011*"msm" + 0.011*"lies" + 0.010*"still" + 0.010*"meeting" + 0.010*"msnbc"
2018-08-16 17:55:54,020 : INFO : topic diff=0.102505, rho=0.102956
2018-08-16 17:55:54,030 : INFO : PROGRESS: pass 29, at document #98000/128681
2018-08-16 17:55:54,258 : INFO : merging changes from 2000 documents into a model of 128681 documents
2018-08-16 17:55:54,371 : INFO : topic #10 (0.050): 0.018*"go" + 0.016*"big" + 0.015*"much" + 0.014*"time" + 0.013*"realjameswoods" + 0.013*"illegal" + 0.012*"usa" + 0.012*"emails" + 0.011*"th" + 0.011*"ste"
2018-08-16 17:55:54,374 : INFO : topic #14 (0.050): 0.019*"fire" + 0.017*"years" + 0.015*"party" + 0.015*"bad" + 

2018-08-16 17:55:56,691 : INFO : topic diff=0.103318, rho=0.102956
2018-08-16 17:55:56,701 : INFO : PROGRESS: pass 29, at document #110000/128681
2018-08-16 17:55:56,938 : INFO : merging changes from 2000 documents into a model of 128681 documents
2018-08-16 17:55:57,058 : INFO : topic #17 (0.050): 0.029*"want" + 0.024*"dont" + 0.022*"texas" + 0.019*"think" + 0.012*"mt" + 0.012*"see" + 0.011*"job" + 0.010*"truth" + 0.010*"read" + 0.010*"found"
2018-08-16 17:55:57,060 : INFO : topic #5 (0.050): 0.027*"back" + 0.024*"maga" + 0.016*"way" + 0.016*"live" + 0.015*"free" + 0.014*"women" + 0.013*"trump" + 0.011*"get" + 0.011*"hate" + 0.011*"speech"
2018-08-16 17:55:57,062 : INFO : topic #9 (0.050): 0.033*"says" + 0.026*"garry" + 0.024*"man" + 0.016*"need" + 0.016*"support" + 0.014*"life" + 0.013*"thats" + 0.012*"right" + 0.012*"ever" + 0.011*"baby"
2018-08-16 17:55:57,064 : INFO : topic #3 (0.050): 0.042*"trump" + 0.026*"charlottesville" + 0.024*"look" + 0.022*"breaking" + 0.021*"arb" + 0.021*

2018-08-16 17:55:59,783 : INFO : topic #8 (0.050): 0.024*"fbi" + 0.022*"lynch" + 0.020*"comey" + 0.015*"chicago" + 0.014*"tarmac" + 0.012*"breaking" + 0.012*"fisa" + 0.012*"dead" + 0.010*"bleachbit" + 0.010*"attack"
2018-08-16 17:55:59,785 : INFO : topic #15 (0.050): 0.047*"pjnet" + 0.017*"rice" + 0.014*"say" + 0.012*"obamacare" + 0.012*"awan" + 0.012*"gop" + 0.011*"trump" + 0.010*"susan" + 0.010*"congress" + 0.010*"every"
2018-08-16 17:55:59,788 : INFO : topic #5 (0.050): 0.111*"maga" + 0.024*"unmasking" + 0.022*"back" + 0.020*"trump" + 0.015*"american" + 0.015*"way" + 0.014*"hate" + 0.012*"free" + 0.012*"women" + 0.011*"live"
2018-08-16 17:55:59,790 : INFO : topic #17 (0.050): 0.031*"dont" + 0.030*"want" + 0.021*"think" + 0.016*"see" + 0.016*"nsa" + 0.015*"texas" + 0.013*"job" + 0.012*"truth" + 0.011*"found" + 0.011*"mt"
2018-08-16 17:55:59,792 : INFO : topic diff=0.098213, rho=0.102956
2018-08-16 17:55:59,802 : INFO : PROGRESS: pass 29, at document #124000/128681
2018-08-16 17:56:00

In [18]:
lda.print_topics()

2018-08-15 14:56:54,958 : INFO : topic #0 (0.100): 0.067*"#" + 0.050*"’" + 0.043*"trump" + 0.018*"“" + 0.018*"”" + 0.014*"‘" + 0.010*"bea" + 0.010*"beai" + 0.010*"liber" + 0.009*"beeth"
2018-08-15 14:56:54,962 : INFO : topic #1 (0.100): 0.016*"bbsp" + 0.012*"bam" + 0.011*"➠" + 0.011*"nsa" + 0.009*"busi" + 0.009*"cant" + 0.009*"join" + 0.008*"america" + 0.008*"patriot" + 0.007*"us"
2018-08-15 14:56:54,968 : INFO : topic #2 (0.100): 0.065*"rtamerica" + 0.026*"rt" + 0.011*"lie" + 0.008*"power" + 0.008*"huma" + 0.006*"god" + 0.006*"new" + 0.005*"school" + 0.005*"charg" + 0.005*"harvey"
2018-08-15 14:56:54,972 : INFO : topic #3 (0.100): 0.064*"#" + 0.020*"trump" + 0.020*"news" + 0.018*"’" + 0.009*"nfl" + 0.009*"break" + 0.007*"new" + 0.007*"show" + 0.007*"via" + 0.007*"call"
2018-08-15 14:56:54,975 : INFO : topic #4 (0.100): 0.128*"#" + 0.017*"antifa" + 0.016*"break" + 0.011*"polic" + 0.011*"demndeb" + 0.011*"charlottesvill" + 0.010*"video" + 0.009*"news" + 0.009*"demdeb" + 0.007*"arrest"
2

[(0,
  '0.067*"#" + 0.050*"’" + 0.043*"trump" + 0.018*"“" + 0.018*"”" + 0.014*"‘" + 0.010*"bea" + 0.010*"beai" + 0.010*"liber" + 0.009*"beeth"'),
 (1,
  '0.016*"bbsp" + 0.012*"bam" + 0.011*"➠" + 0.011*"nsa" + 0.009*"busi" + 0.009*"cant" + 0.009*"join" + 0.008*"america" + 0.008*"patriot" + 0.007*"us"'),
 (2,
  '0.065*"rtamerica" + 0.026*"rt" + 0.011*"lie" + 0.008*"power" + 0.008*"huma" + 0.006*"god" + 0.006*"new" + 0.005*"school" + 0.005*"charg" + 0.005*"harvey"'),
 (3,
  '0.064*"#" + 0.020*"trump" + 0.020*"news" + 0.018*"’" + 0.009*"nfl" + 0.009*"break" + 0.007*"new" + 0.007*"show" + 0.007*"via" + 0.007*"call"'),
 (4,
  '0.128*"#" + 0.017*"antifa" + 0.016*"break" + 0.011*"polic" + 0.011*"demndeb" + 0.011*"charlottesvill" + 0.010*"video" + 0.009*"news" + 0.009*"demdeb" + 0.007*"arrest"'),
 (5,
  '0.011*"gun" + 0.008*"fire" + 0.008*"john" + 0.008*"say" + 0.007*"us" + 0.006*"victim" + 0.006*"googl" + 0.006*"jame" + 0.005*"russian" + 0.005*"ask"'),
 (6,
  '0.026*"beer" + 0.014*"rt" + 0.012

### Vis

In [89]:
word2id = {v : k for k, v in id2word.items()}

In [100]:
followers_data =  pyLDAvis.gensim.prepare(lda, corpus, g_dict)
pyLDAvis.display(followers_data)

/Users/bradpanzarella/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))
